In [8]:
import configparser
import csv
import datetime
import os
import platform
import sys
import time
from typing import Union, Optional, List, Dict, Tuple, TextIO, Any
import bs4
import pandas
import requests

url_oc: str = "https://www.nseindia.com/option-chain"
url_index: str = "https://www.nseindia.com/api/option-chain-indices?symbol="
url_stock: str = "https://www.nseindia.com/api/option-chain-equities?symbol="
url_symbols: str = "https://www.nseindia.com/products-services/equity-derivatives-list-underlyings-information"

session: requests.Session = requests.Session()
cookies: Dict[str, str] = {}

headers: Dict[str, str] = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                          'like Gecko) Chrome/80.0.3987.149 Safari/537.36',
            'accept-language': 'en,gu;q=0.9,hi;q=0.8',
            'accept-encoding': 'gzip, deflate, br'}

index = "NIFTY"
url: str = url_index + index
request = session.get(url_oc, headers=headers, timeout=5)
cookies = dict(request.cookies)
response = session.get(url, headers=headers, timeout=5, cookies=cookies)
if response is not None:
            try:
                json_data = response.json()
            except Exception as err:
                print(response)
                print(err, sys.exc_info()[0], "2")
                json_data = {}
else:
    json_data = {}
    
avl_dates = []
for dates in json_data['records']['expiryDates']:
    avl_dates.append(dates)
    
# get_data_refresh
request: Optional[requests.Response] = None
response: Optional[requests.Response] = None
url: str = url_index + index
try:
    response = session.get(url, headers=headers, timeout=5, cookies=cookies)
    if response.status_code == 401:
        session.close()
        session = requests.Session()
        request = session.get(url_oc, headers=headers, timeout=5)
        cookies = dict(request.cookies)
        response = session.get(url, headers=headers, timeout=5, cookies=cookies)
        print("reset cookies")
except Exception as err:
    print(request)
    print(response)
    print(err, sys.exc_info()[0], "4")
    try:
        session.close()
        session = requests.Session()
        request = session.get(url_oc, headers=headers, timeout=5)
        cookies = dict(request.cookies)
        response = session.get(url, headers=headers, timeout=5, cookies=cookies)
        print("reset cookies")
    except Exception as err:
        print(request)
        print(response)
        print(err, sys.exc_info()[0], "5")
if response is not None:
    try:
        json_data: Any = response.json()
    except Exception as err:
        print(response)
        print(err, sys.exc_info()[0], "6")
        json_data = {}
else:
    json_data = {}
    
# get_dataframe
try:
    response, json_data = response, json_data
except TypeError:
    print("Error")
if response is None or json_data is None:
    print("Response Null")

pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', 400)

df: pandas.DataFrame = pandas.read_json(response.text)
df = df.transpose()

expiry_date = '13-Oct-2022'
ce_values: List[dict] = [data['CE'] for data in json_data['records']['data'] if
                            "CE" in data and data['expiryDate'].lower() == expiry_date.lower()]
pe_values: List[dict] = [data['PE'] for data in json_data['records']['data'] if
                            "PE" in data and data['expiryDate'].lower() == expiry_date.lower()]

points: float = pe_values[0]['underlyingValue']
if points == 0:
    for item in pe_values:
        if item['underlyingValue'] != 0:
            points = item['underlyingValue']
            break
ce_data_f: pandas.DataFrame = pandas.DataFrame(ce_values)
pe_data_f: pandas.DataFrame = pandas.DataFrame(pe_values)

columns_ce: List[str] = ['openInterest', 'changeinOpenInterest', 'totalTradedVolume', 'impliedVolatility',
                            'lastPrice',
                            'change', 'bidQty', 'bidprice', 'askPrice', 'askQty', 'strikePrice']
columns_pe: List[str] = ['strikePrice', 'bidQty', 'bidprice', 'askPrice', 'askQty', 'change', 'lastPrice',
                            'impliedVolatility', 'totalTradedVolume', 'changeinOpenInterest', 'openInterest']
ce_data_f = ce_data_f[columns_ce]
pe_data_f = pe_data_f[columns_pe]
merged_inner: pandas.DataFrame = pandas.merge(left=ce_data_f, right=pe_data_f, left_on='strikePrice',
                                                right_on='strikePrice')
merged_inner.columns = ['Open Interest', 'Change in Open Interest', 'Traded Volume', 'Implied Volatility',
                        'Last Traded Price', 'Net Change', 'Bid Quantity', 'Bid Price', 'Ask Price',
                        'Ask Quantity', 'Strike Price', 'Bid Quantity', 'Bid Price', 'Ask Price',
                        'Ask Quantity', 'Net Change', 'Last Traded Price', 'Implied Volatility',
                        'Traded Volume', 'Change in Open Interest', 'Open Interest']
current_time: str = df['timestamp']['records']

merged_inner


,Open Interest,Change in Open Interest,Traded Volume,Implied Volatility,Last Traded Price,Net Change,Bid Quantity,Bid Price,Ask Price,Ask Quantity,Strike Price,Bid Quantity,Bid Price,Ask Price,Ask Quantity,Net Change,Last Traded Price,Implied Volatility,Traded Volume,Change in Open Interest,Open Interest
0,0,0,0,0.00,0.00,0.00,1500,2449.80,2515.05,1500,14650,2700,0.15,0.25,1800,-0.25,0.15,93.65,3398,-589,2330
1,1,0,0,0.00,3000.00,0.00,1500,2395.85,2465.30,1500,14700,5400,0.15,0.20,5600,-0.20,0.15,91.76,3237,-541,2807
2,0,0,0,0.00,0.00,0.00,1500,2348.75,2415.75,1500,14750,400,0.15,0.50,1100,-0.15,0.20,91.96,736,33,201
3,0,0,0,0.00,0.00,0.00,1500,2296.25,2364.35,1500,14800,400,0.25,0.30,400,-0.10,0.30,93.15,184,-11,213
4,0,0,0,0.00,0.00,0.00,1500,2251.20,2313.35,1500,14850,9150,0.20,0.40,550,-0.20,0.40,93.52,400,41,216
5,0,0,0,0.00,0.00,0.00,1500,2197.05,2265.85,1500,14900,13700,0.20,0.30,100,-0.30,0.20,86.19,857,-94,258
6,1,0,0,0.00,2342.50,0.00,1500,2151.40,2220.65,1500,14950,2650,0.25,0.45,1800,-0.35,0.25,85.87,1209,39,133
7,22,1,4,0.00,1986.15,-14.85,1500,2099.60,2170.50,1500,15000,154350,0.20,0.25,24600,-0.25,0.20,82.36,56300,1307,30065
8,0,0,0,0.00,0.00,0.00,1500,2050.40,2117.55,1500,15050,500,0.30,0.45,1800,-0.30,0.30,83.29,287,18,141
9,0,0,0,0.00,0.00,0.00,1500,2000.60,2068.00,1500,15100,17550,0.20,0.35,400,-0.25,0.20,78.55,1144,-23,514
